
### **Retrieval-Augmented Generation (RAG)**

Retrieval-Augmented Generation (RAG) is a powerful approach that combines information retrieval with text generation. Instead of relying solely on a language model’s internal knowledge, RAG retrieves relevant documents from an external source before generating responses. This enhances accuracy, reduces hallucinations, and provides up-to-date information.

In this lesson, we will implement RAG using **Together AI’s API** for both **document embeddings** and **text generation**. To follow along, get an API key from [Together AI](https://api.together.ai/).

---

### Setting Up the Environment

Before we begin, ensure you have the necessary libraries installed:

In [ ]:
%pip install pandas langchain chromadb langchain-together 

**Import the required libraries:**

- `pandas`: Used to load and manipulate the dataset.
- `uuid4`: Generates unique document IDs.
- `TogetherEmbeddings`: Embeds text into vector representations.
- `Chroma`: Stores and retrieves embedded documents efficiently.
- `ChatTogether`: Uses an AI model for text generation.
- `ChatPromptTemplate` and `StrOutputParser`: Create structured prompts and extract responses.
- `dotenv` :  This helps to load the environment variables like API Keys.

In [1]:
from langchain_chroma import Chroma
import pandas as pd
from uuid import uuid4
import os
import chromadb
import pathlib
from langchain_together import ChatTogether
from langchain_together import TogetherEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

**Loading the Dataset**


For this tutorial, we’ll use a CSV file containing data about MSMEs (Micro, Small, and Medium Enterprises) in Nigeria.

In [2]:
# Load the dataset
msme = pd.read_csv("msme.csv")

When storing and retrieving documents in a Retrieval-Augmented Generation (RAG) system, we need more than just the text. Metadata and ID play a crucial role in managing, organizing, and retrieving relevant documents effectively.

In [3]:
documents = []
metadatas = []
ids = []

for index, row in msme.iterrows():    
    texts = str(row["Content"])
    title = row["Title"]
    sources = row["Sources"]
        
    content = texts
    metadata = {"Source":sources ,  "doc_title": title}
    id = f"{title}-{uuid4()}"

    documents.append(content)
    metadatas.append(metadata)
    ids.append(id)



In [37]:
#use this to comfirm the number documents to embed 
print(f"Total Documents to be embedded {len(documents)}\n")

Total Documents to be embedded 14



In this tutorial, we are using the document title and source as metadata.

Also, we will be using the title and  UUID as as the ID.  
  
  - `Content`: The document’s main text.
  - `Title`: The document title.
  - `Sources`: source / links to each documents.

  - `documents`: Holds document texts.
  - `metadatas`: Metadata provides additional information about the document and also helps in filtering and refining search results.
  - `ids`: Ensures each document has a unique identifier, useful for tracking and retrieval.


This loads the environment variables from a file named credentials.env, which contains the together API key.

In [4]:
load_dotenv(dotenv_path="./credentials.env")

True

**Why Should You Avoid Exposing API Keys?**

If an API key is hardcoded in your script and gets exposed, malicious users can exploit it, leading to excessive API usage, financial loss

In [5]:
together_api = os.environ["TOGETHER_API_KEY"]

The code above Retrieves the API key stored in the environment variable TOGETHER_API_KEY.

Using os.environ ensures that the API key remains hidden and can be accessed securely.

**Setting Up ChromaDB for embeddings Storage**

chroma_dir: Gets the current working directory where the notebook is located.

chroma_path: Creates a folder named chroma_store to store the vector embeddings.

In [6]:
#chroma_dir: Gets the current working directory where the notebook is located.
#chroma_path: Creates a folder named chroma_store to store the vector nembeddings.

chroma_dir = pathlib.Path().resolve()
chroma_path = f"{chroma_dir}\\chroma_store"


You can use both persist_directory(path) or client for a ChromaDB Vector store.

PersistentClient manages multiple collections in a single directory (chroma_path). It allows multiple collections to share the same persistent database.

While the persist_directory(path) saves embeddings for a specific collection.

In [ ]:
# client = chromadb.PersistentClient(path= chroma_path)

- ** For this tutorial We are using the `m2-bert-80M-8k-retrieval` embedding model** to convert documents into vector embeddings.
- ** you can check out More embedding models are  on Together AI (https://together.ai/)).

In [7]:
# Initialize the embedding model
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval",
    api_key=together_api
)

- Creates a **vector database (Chroma)** for MSME documents.
- Adds **embedded texts**, **metadata**, and **IDs** to the database.

In [8]:
# Initialize ChromaDB
msmevdb = Chroma(
    collection_name="MSME",
    embedding_function=embeddings,
    persist_directory=chroma_path
)

# Add documents to the vector database
msmevdb.add_texts(
    texts=documents,
    metadatas=metadatas,
    ids=ids
)

["**Introduction 1. Definition and Importance of MSMEs**    - Definitions according to Nigerian policies (e.g., SMEDAN criteria).    - Role of MSMEs in Nigeria's economy (employment, GDP contribution, innovation).-86d95986-e19a-4534-ad16-b1acf31ff800",
 '**Understanding MSMEs in Nigeria** 1. **Overview of the MSME Sector**    - Statistical data (number of MSMEs, sectoral distribution, urban vs. rural).    - Key industries and opportunities.-d5c70d24-80f3-41cc-8af7-86a7d503e314',
 '**Understanding MSMEs in Nigeria**  2. **Challenges Faced by MSMEs**    - Access to financing.    - Regulatory hurdles.    - Infrastructure deficits.    - Competition and market access.-880f4d1e-e1d3-4bbf-9ca7-01266df8e1d4',
 '**Understanding MSMEs in Nigeria**3. **Policies and Support Frameworks**    - Government initiatives (e.g., SMEDAN, CBN interventions).    - Available grants and loans (e.g., AGSMEIS, NIRSAL MFB).    - International support programs.-cfc0005b-cc9d-443c-82da-7cff3d854721',
 '**Starting a

In [ ]:
#comfirm if the embedded documents is equal to the total number of documents
print(len(msmevdb.get()["documents"]) )

14


### The Retriever
Retrieving Documents for a Query

In [9]:
# Maximal Marginal Relevance (MMR) for diverse and relevant results.
msme_retriever = msmevdb.as_retriever(search_type="mmr")

k defines the final number of documents that the retriever will return. use for concise and relevant results instead of retrieving too much data.

fetch_k determines how many documents are initially retrieved before filtering.

In [ ]:
msme_retriever = msmevdb.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 10})

Retrieves related documents based on `question`

In [28]:
question = "How long does a business name reservation last in Nigeria?"
msme_retriever.invoke(question)

[Document(metadata={'Source': 'https://simplebks.com/blog/challenges-and-solutions-facing-smes-in-nigeria-today-a-comprehensive-guide/, https://www.nafdac.gov.ng/wp-content/uploads/Publications/Laws/NATIONAL-POLICY-ON-MSMEs-x.pdf, https://omaplex.com.ng/improving-the-nigerian-economy-through-msmes/#google_vignette, https://proshare.co/articles/empowering-nigerias-small-and-medium-enterprises-smes-overcoming-challenges-for-economic-prosperity?menu=MSMEs&classification=Read&category=Enterpreneurship', 'doc_title': '**Understanding MSMEs in Nigeria**  2. **Challenges Faced by MSMEs**    - Access to financing.    - Regulatory hurdles.    - Infrastructure deficits.    - Competition and market access.'}, page_content='CHALLENGES CONFRONTING MSMEs IN NIGERIA The Micro, Small and Medium Enterprises (MSMEs) have been known, in both developed and developing nations, to be incontrovertible contributors to employment generation, wealth creation and poverty alleviation. It is on this premise that s

**Generating Responses Using Llama 3.1 model**

- Let us use **Meta Llama 3.1 (8B)** for response generation.
- **`temperature= 0` ensures deterministic outputs.**
- You can try Other chat models are available on **Together AI (https://together.ai/))**.

In [11]:
#initialize the chatmodel
chatmodel = ChatTogether(
    api_key=together_api,
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    temperature=0
)

### **Prompting the LLM**

Creates a prompt template (prompt) with a system prompt and placeholders for the context and user question

In [21]:
#using from template method
prompt = ChatPromptTemplate.from_template(
    """You are a business consultant providing insights on MSMEs in Nigeria.
You will be provided with the context: {context} to answer the user's question.
The context includes sections on understanding, starting, growing, and sustaining MSMEs, policies, and industry-specific information.
Provide a comprehensive response.
Include relevant sources or links from the context in your response at the end of each answer, include a statement: "To read more, check out this link: [insert link]."
Avoid unnecessary or unrelated details. Format the text output clearly and professionally in an HTML format.
question: {question}""")




Initializes the chat completion chat model (llm) to use for generating responses

Creates a simple output parser (parse_output) to parse the LLM output as a string

Chains all the above components using LangChain’s pipe ( | ) notation to create a simple RAG workflow (rag_chain)

In [ ]:
question = "How long does a business name reservation last in Nigeria?"

#retrieve the document
get_doc = msme_retriever.invoke(question)
# Prepare the input for the chain
input = {"context": get_doc, "question": question}

# Create the chain
chain = prompt | chatmodel | StrOutputParser()

answer = chain.invoke(input)
print(answer)

<h2>Business Name Reservation Duration in Nigeria</h2>

In Nigeria, a business name reservation is valid for 60 days from the date of payment. This means that once you pay for the reservation, you have 60 days to complete the registration process and obtain a Certificate of Incorporation.

If you fail to complete the registration process within the 60-day period, the business name will be released back to the public domain, and you will need to re-reserve the name.

It's essential to note that the 60-day period is a one-time extension, and you cannot request an extension beyond this period.

To read more, check out this link: <a href="https://www.cac.gov.ng/">Corporate Affairs Commission (CAC) Website</a>.


**Using from messages method**

- Separates **system instructions and user input** into structured messages.
- Can be adapted for other structured conversations.


In [33]:
# Define the prompt using from_messages
prompt_messages = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a business consultant providing insights on MSMEs in Nigeria.
            You will be provided with the context: {context} to answer the user's question.
            The context includes sections on understanding, starting, growing, and sustaining MSMEs, policies, and industry-specific information.
            Provide a comprehensive response.
            Include relevant sources or links from the context in your response at the end of each answer, include a statement: "To read more, check out this link: [insert link]."
            Avoid unnecessary or unrelated details. Format the text output clearly and professionally in an HTML format."""
        ),
        ("human", "question: {question}"),
    ]
)

# Prepare the input for the chain
input = {"context": get_doc, "question": question}
# Create the chain
chain = prompt_messages | chatmodel | StrOutputParser()
# Generate and print the answer
result = chain.invoke(input)
print(result)

<h2>Business Name Reservation in Nigeria</h2>

In Nigeria, a business name reservation is valid for 30 days from the date of reservation. This means that a business owner has 30 days to register their business name with the Corporate Affairs Commission (CAC) after reserving it.

<h3>Why is Business Name Reservation Important?</h3>

Business name reservation is an essential step in the process of registering a business in Nigeria. It allows a business owner to secure their desired business name and prevent others from using it. This is particularly important in Nigeria, where many businesses operate in the same industry or sector.

<h3>How to Reserve a Business Name in Nigeria</h3>

To reserve a business name in Nigeria, a business owner can follow these steps:

1. Visit the CAC website and fill out the business name reservation form.
2. Pay the required fee for business name reservation.
3. Submit the form and wait for confirmation from the CAC.

<h3>What Happens if I Don't Register My

## **Other Retriever Techniques**

### Contextual Compression Retriever

This method filters and compresses retrieved documents to retain only the most relevant parts, Focuses on the most useful information and improving precision.



First, import the necessary libraries for compression and retrieval.

In [35]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA


 Create a Base Retriever

we will use the same data set and Retriever

In [34]:
# Create a retriever
retriever = msmevdb.as_retriever()

here we use an LLM to analyze and compress the retrieved documents, keeping only the most relevant parts

In [ ]:
chatmodel = ChatTogether(
    api_key=together_api,
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    temperature=0
)
compressor = LLMChainExtractor.from_llm(chatmodel)

Combine the base retriever with the compressor to create a Contextual Compression Retriever.

In [37]:
#Combine the retriever with the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)


Set up a RetrievalQA chain to generate answers using the compressed retriever. 

You can choose to add your prompt or use the prebuilt prompt.

In [ ]:
# Create a QA chain with the compressed retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=chatmodel ,
    retriever=compression_retriever,
    chain_type_kwargs={"prompt": prompt},  
    return_source_documents=True
)


In [45]:
# Define the query
query = "How can I register my farm business in Nigeria?"

# Invoke the QA chain
result = qa_chain.invoke({"query": query})

# Print the result
print("Answer:", result["result"])
# print("Source documents:", result["source_documents"])

Answer: <html>
  <body>
    <h1>Registering Your Farm Business in Nigeria</h1>
    <p>As a business consultant, I'd be happy to guide you through the process of registering your farm business in Nigeria.</p>
    <h2>Step 1: Choose the Right Business Structure</h2>
    <p>MSMEs in Nigeria can register under various parts of the Companies and Allied Matters Act (CAMA). It is usually advised that SMEs register under Part B of the CAMA.</p>
    <h2>Step 2: Register with the Corporate Affairs Commission (CAC)</h2>
    <p>The CAC is responsible for regulating the formation, management, and winding up of companies in Nigeria. You can register your farm business with the CAC by following these steps:</p>
    <ol>
      <li>Visit the CAC website at <a href="https://www.cac.gov.ng/">www.cac.gov.ng</a> and click on "Register a Business".</li>
      <li>Fill out the registration form and provide the required documents, including your business name, address, and details of the business owners.</li>

## Parent Document Retriever 


When preparing documents for LLMs:

Small chunks improve retrieval quality.
Large chunks maintain context for better generation.
Simple strategies like fixed or recursive splitting can't balance both. 

**Parent document retrieval solves this by:**
Embedding small chunks for retrieval.
Fetching larger chunks or source documents for context.

This ensures the LLM gets complete context, improving response quality. Useful for tasks needing expanded context.

#### import the necessary libraries 

We will require the following libraries for this method:

In [1]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

We will be using the Parent Document Retriever library from LangChain.
  The easiest way to use your data with LangChain features is by converting them into LangChain document 

** This document consist of two attributes—page_content and metadata.**

In [5]:
#we will use the same MSME data set
msme = pd.read_csv("msme.csv")
msme_documents= []

for index, row in msme.iterrows():    
    # Extract content and metadata fields
    texts = str(row["Content"])
    title = str(row["Title"])
    sources = str(row["Sources"])
    
    # Create metadata dictionary
    metadata = {"source": sources, "doc_title": title}
    # Create Document object with content and metadata
    doc = Document(page_content=texts, metadata=metadata)
    
    msme_documents.append(doc)

print(f"Total Documents to be embedded: {len(msme_documents)}\n")

Total Documents to be embedded: 14



In [6]:
#This is to divides the documents into smaller chunks
split_msme = RecursiveCharacterTextSplitter(chunk_size=500)

Now, let's create a vector store for storing smaller chunks and an InMemoryStore to store the parent documents.

The InMemoryStore functions as a key-value pair data structure, where:

Each key is a unique UUID assigned to a parent document.

Each value is the actual text content of the corresponding parent document.

This structure ensures that while the program is running, the parent documents remain accessible in memory, allowing efficient retrieval and reconstruction of full documents when needed.


In [13]:
msme_vectorestore = Chroma(collection_name="msme_documents", embedding_function=embeddings)
# This store will retain parent documents for retrieval
store = InMemoryStore()

Initialize the Parent Document Retriever

In [14]:
retriever = ParentDocumentRetriever(
    vectorstore = msme_vectorestore , 
    docstore=store, 
    child_splitter=split_msme
)

Add Documents to Retriever

In [15]:
retriever.add_documents(msme_documents)

After adding, we can see there are 14 keys in the store. So 14 large documents have been added.

In [16]:
len(list(store.yield_keys()))

14

 This will only returns the small chunks

 Search for relevant document chunks using the vector store

In [17]:
sub_docs = msme_vectorestore .similarity_search("What is the first step in setting up a business in Nigeria?")
sub_docs

[Document(metadata={'doc_id': '550bdf46-2afd-48fc-8e34-a903e36330a0', 'doc_title': '**Financial Management for MSMEs** 2. **Tax Obligations and Benefits**    - Tax filing processes for MSMEs in Nigeria.    - Available tax incentives and how to leverage them.', 'source': 'https://www.youtube.com/watch?v=PuXaGOzQfKQ'}, page_content='so you can focus on what you do best—growing your business.   Small and Medium Enterprises (SMEs) are the backbone of Nigeria’s economy, driving innovation, creating jobs, and contributing significantly to economic growth. Recognizing the vital role played by SMEs, the Nigeria Finance Act of 2019 introduces a range of tax incentives to support their development and foster a conducive business environment. As a leading accounting firm in Nigeria, we explore the tax incentives provided under the'),
 Document(metadata={'doc_id': 'ea2c2431-9d17-4b62-a19e-9e0d1ebf767c', 'doc_title': '**Starting an MSME in Nigeria** 2. **Business Registration and Legal Requirements

: Retrieve Relevant Parent Documents

In [19]:
retrieved_docs = retriever.get_relevant_documents("What is the first step in setting up a business in Nigeria?")

print(retrieved_docs[0].page_content)

Taxes for small businesses in Nigeria are usually not paid enough attention. Many small businesses either forget or purposely skip tax payments in their budgets.  Taxes are what businesses pay the government for operating. Whether you run a sole proprietorship, partnership, limited liability company, or corporation, you must follow your country’s tax rules. Taxes fund federal, state, and local government activities, so if your business is registered, a portion of your profits goes to the government.  Understanding Taxes in Nigeria All employees, business owners, non-residents earning income in Nigeria, and companies operating there must pay taxes. Federal, state, and local governments manage Nigeria’s tax system.  Federal Taxes: Companies Income Tax (CIT), Value Added Tax (VAT), capital gains tax, stamp duty, education tax, petroleum profit tax, etc. State Taxes: Personal Income Tax, Business Premises Tax, development levy, etc. Local Taxes: Various levies and rates. Types of Tax for S

### Generate Response 

we will use the same prompt above

In [25]:
question = "How long does a business name reservation last in Nigeria?"

#retrieve the document
retrieved_docs = retriever.get_relevant_documents(question)

# Prepare the input for the chain
input = {"context": retrieved_docs, "question": question}

# Create the chain
chain = prompt | chatmodel | StrOutputParser()

answer = chain.invoke(input)
print(answer)

<h2>Business Name Reservation Duration in Nigeria</h2>

In Nigeria, a business name reservation lasts for 60 days. This means that once a business name is reserved, the applicant has 60 days to complete the registration process and obtain the Certificate of Registration.

According to the Companies and Allied Matters Act (CAMA) 2020, Section 1(1) states that "Every individual, firm or Corporation having a place of business in Nigeria and carrying on business under a business name shall be registered in the manner provided in this Part if...". Additionally, Section 2(1) states that "Every individual, firm or company required under this Act to be registered shall, within 28 days after the individual, firm or corporation commences the business in respect of which registration is required, furnish to the Registrar at the registry in the State in which the principal place of business of the individual, firm or company is situated, a statement in writing in the prescribed form...".

This imp

### Multiple Query Retriever

This method enhances document retrieval by generating multiple variations of a user query, increasing the likelihood of retrieving relevant documents from a vector database

 We will prompt the LLM to create five different variations of the user’s query.
Helps overcome limitations of distance-based similarity search, ensuring that relevant documents aren't missed due to minor phrasing differences.

In [28]:

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple type of the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. Do not add explanation or numbers to the question, only 
generate the questions.
Provide these alternative questions separated by newlines Original question: {question}"""
multiple_query_prompt = ChatPromptTemplate.from_template(template)

alternative versions of the query, which are split into individual questions using split("\n") to create a list.

Since the LLM generates multiple queries as a block of text with line breaks, we split the text into individual lines to get distinct query variations.

Using .strip() prevents storing empty or whitespace-only strings in the final output.

In [ ]:
question = "How long does a business name reservation last in Nigeria?"

multiple_queries = multiple_query_prompt | chatmodel | StrOutputParser() | (lambda x: [q for q in x.split("\n") if q.strip()])
multiple_answer = multiple_queries.invoke({"question" : question})
print(multiple_answer)

['How long does a business name reservation typically last in Nigeria?', 'What is the standard duration of a business name reservation in Nigeria?', 'How long is a business name reservation valid for in Nigeria?', 'What is the average duration of a business name reservation in Nigeria?', 'How long does a business name reservation remain active in Nigeria?']


Define the Retriever

In [32]:
msme_retriever = msmevdb.as_retriever()

Get Unique Documents


Let us Convert the documents into JSON strings using dumps() and also remove duplicates using (set), and convert them back to objects using loads().

This is to ensure the retrieved documents are unique.

In [ ]:
from langchain.load import dumps, loads
def unique_doc (documents): 
    unique_doc = list(set(dumps(doc)for doc in documents))
    return [loads(doc) for doc in unique_doc]

Retrieving Documents Using Multiple Queries

With .map(), msme_retriever processes each query separately, retrieving relevant documents for each one instead of handling all queries at once.

In [34]:
retrieval_chain = multiple_queries | msme_retriever.map() | unique_doc

Answer Generation Using Retrieved Documents

 We pass all the retrieved documents as context along with the original question to the llm

In [36]:

prompt = ChatPromptTemplate.from_template(
    """You are a business consultant providing insights on MSMEs in Nigeria.
You will be provided with the context: {context} to answer the user's question.
The context includes sections on understanding, starting, growing, and sustaining MSMEs, policies, and industry-specific information.
Provide a detailed and comprehensive response.
Include relevant sources or links from the context in your response. At the end of each answer, include a statement: "To read more, check out this link: [insert link]."
Avoid unnecessary or unrelated details. Format the text output clearly and professionally in an HTML format.
question: {question}""")

multiple_query_input = ({"context" : retrieval_chain, "question": question})

chain = prompt| chatmodel| StrOutputParser()

answer = chain.invoke(multiple_query_input)
print(answer)  

<h2>Business Name Reservation in Nigeria</h2>

<p>The duration of a business name reservation in Nigeria is a crucial aspect for Micro, Small, and Medium Enterprises (MSMEs) to consider when registering their businesses.</p>

<h3>Understanding Business Name Reservation in Nigeria</h3>

<p>Business name reservation in Nigeria is a process where an individual or organization can reserve a unique name for their business before officially registering it with the Corporate Affairs Commission (CAC). This is done to prevent others from using the same name.</p>

<h3>Duration of Business Name Reservation in Nigeria</h3>

<p>According to the Corporate Affairs Commission (CAC), a business name reservation in Nigeria is valid for 60 days from the date of application. This means that if you apply for a business name reservation, you have 60 days to complete the registration process and pay the required fees.</p>

<h3>Consequences of Expiring Business Name Reservation in Nigeria</h3>

<p>If the busi

### Exercises: Hands-On RAG


Build a RAG engine and Implement two different retrieval methods using a chosen dataset of your choice 

Compare and analyze each retrieval performance
